In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import os
import sys
from importlib import reload
from scipy.stats import zscore
import scipy.io
from scipy.interpolate import make_interp_spline, BSpline
from scipy.signal import savgol_filter as savgol 
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import leidenalg
import umap
import time
import gget
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from scipy.signal import savgol_filter
import scipy
import textwrap

from pydmd import DMD

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# local imports
sys.path.append("../../python/")
sys.path.append("../")

import nb_util as nb
import utils as ut
reload(ut)

import hasnain_DMD
reload(hasnain_DMD)

2024-01-09 09:15:26.394676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-09 09:15:26.501309: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-09 09:15:26.505734: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-09 09:15:26.505749: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

<module 'hasnain_DMD' from '/home/cstansbu/git_repositories/DMD_gene/notebooks/2015/../../python/hasnain_DMD.py'>

# Load Data

In [2]:
data_path = f"/nfs/turbo/umms-indikar/shared/projects/cell_cycle/data/RNA_pipeline_ouputs/countMatrix/counts.raw.txt"
gene_path = f"/nfs/turbo/umms-indikar/shared/projects/cell_cycle/data/RNA_pipeline_ouputs/references/geneTable.csv"

""" Load the raw expression """
df = pd.read_csv(data_path, index_col=0)

# remove MT and ribosomal genes
all_genes = df.index.to_list()
mt_genes = [x for x in all_genes if x.startswith('MT-')]
rp_genes = [x for x in all_genes if x.startswith('RP')]

print(f"{df.shape=}")
df = df.drop(mt_genes) # drop MT genes
df = df.drop(rp_genes) # drop ribosomal genes
print(f"{df.shape=}")

# rewrite the list without MT genes
gene_names = df.index.to_list()

print(f"{len(all_genes)=} {len(mt_genes)=} {len(gene_names)=}")

""" Load gene lengths """
gf = nb.getGeneLengths(gene_path, gene_names)
print(f"{gf.shape=}")


df.head()

df.shape=(19393, 18)
df.shape=(19235, 18)
len(all_genes)=19393 len(mt_genes)=13 len(gene_names)=19235


/tmp/ipykernel_1020159/4036917658.py:23: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  gf = nb.getGeneLengths(gene_path, gene_names)


gf.shape=(19235, 2)


,S1a,S1b,S2a,S2b,S3a,S3b,S4a,S4b,S5a,S5b,S6a,S6b,S7a,S7b,S8a,S8b,S9a,S9b
geneName,,,,,,,,,,,,,,,,,,
A1BG,12,5,5,9,2,6,7,3,6,5,5,5,4,3,4,5,5,3
A1CF,0,0,0,0,0,0,0,0,0,0,0,1,2,0,2,0,0,0
A2M,0,0,0,2,0,0,0,0,0,0,1,0,1,0,0,0,0,2
A2ML1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Choose Rank = 7; TPM > 0.5

In [3]:
target = 1e6
threshold = 0.5
rank = 7
tpm = nb.TPM(df, gf, target=target)

# get highly expressed genes
tpm_dist = tpm.mean(axis=1)
mask = (tpm_dist > threshold)
high_exp_genes = tpm_dist[mask].index.to_list()    

# filter and convert to fold changes
d = tpm[tpm.index.isin(high_exp_genes)]
dmd_data = nb.data2DMD(d) 
print(f"{dmd_data.shape=}")


dmd_data.shape=(8112, 8, 2)


In [4]:
d.head()

,S1a,S1b,S2a,S2b,S3a,S3b,S4a,S4b,S5a,S5b,S6a,S6b,S7a,S7b,S8a,S8b,S9a,S9b
geneName,,,,,,,,,,,,,,,,,,
A4GALT,0.728128,0.629071,0.472031,0.515156,0.523757,0.548982,0.479790,0.529386,0.435893,0.419155,0.540279,0.637680,0.581729,0.572130,0.459156,0.325065,0.631822,0.533986
AAAS,2.684803,2.711166,3.287762,3.548200,3.284197,3.295556,2.398563,2.221554,2.882785,2.674720,2.995194,2.933435,2.426814,2.348625,3.348397,3.501258,2.842093,3.132419
AACS,0.859995,0.822024,0.818799,0.730976,0.659868,0.649130,0.814887,0.732596,0.828100,0.914619,0.830217,0.809711,0.742738,0.812148,0.755614,0.816477,0.740497,0.750778
AAGAB,1.259756,1.273046,1.436806,1.364192,1.258220,1.232621,1.310217,1.344992,1.393150,1.288104,1.146511,1.149491,1.063345,1.098117,0.956624,0.961782,0.983712,0.972340
AAMP,15.521382,15.625681,15.991591,15.052069,13.866394,13.913058,13.265243,13.408798,13.598188,13.686900,13.874603,14.853579,14.027852,12.816619,14.111038,14.293969,13.898976,12.415323


In [6]:
a = [x for x in d.columns if "a" in x]


dfa = nb.getFCFrame(d, a)
scaler = StandardScaler()
dfa_scaled = scaler.fit_transform(dfa)


In [7]:
dfa.head()

,S2a,S3a,S4a,S5a,S6a,S7a,S8a,S9a
geneName,,,,,,,,
A4GALT,0.851807,0.881738,0.856296,0.830895,0.891299,0.915285,0.844356,0.944271
AAAS,1.163634,1.162667,0.922319,1.053729,1.084236,0.929986,1.180090,1.042686
AACS,0.977851,0.892404,0.975748,0.982852,0.983990,0.936958,0.943881,0.935753
AAGAB,1.078349,0.999320,1.022330,1.059030,0.949886,0.913083,0.865856,0.877843
AAMP,1.028461,0.899828,0.863441,0.883594,0.900324,0.909600,0.914635,0.901800


In [9]:
dfa_scaled_df = pd.DataFrame(dfa_scaled, index=dfa.index, columns=dfa.columns)
dfa_scaled_df.head()

,S2a,S3a,S4a,S5a,S6a,S7a,S8a,S9a
geneName,,,,,,,,
A4GALT,-0.556981,-0.262203,-1.012602,-0.557596,-0.240148,-0.499629,-0.498289,-0.121191
AAAS,0.467285,0.338691,-0.478986,0.176310,0.183669,-0.384847,0.554739,0.080604
AACS,-0.142958,-0.239389,-0.047155,-0.057124,-0.036536,-0.330409,-0.186129,-0.138656
AAGAB,0.187148,-0.010700,0.329335,0.193769,-0.111451,-0.516817,-0.430854,-0.257398
AAMP,0.023279,-0.223511,-0.954855,-0.384032,-0.220322,-0.544011,-0.277859,-0.208277


In [12]:
dfa.mean(axis=1)

geneName
A4GALT    0.876993
AAAS      1.067418
AACS      0.953680
AAGAB     0.970712
AAMP      0.912710
            ...   
ZXDC      0.908663
ZYG11A    1.078878
ZYX       0.933164
ZZEF1     0.965133
ZZZ3      1.056875
Length: 8112, dtype: float64

In [13]:
dfa.std(axis=1)

geneName
A4GALT    0.038680
AAAS      0.101452
AACS      0.032277
AAGAB     0.081278
AAMP      0.049462
            ...   
ZXDC      0.026398
ZYG11A    0.085527
ZYX       0.126955
ZZEF1     0.036732
ZZZ3      0.021477
Length: 8112, dtype: float64

In [14]:
(0.85 - 0.87) / 0.038

-0.5263157894736847